# SGLang Frontend Language

SGLang frontend language can be used to define simple and easy prompts in a convenient, structured way.

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

In [1]:
import requests
import os

from sglang import assistant_begin, assistant_end
from sglang import assistant, function, gen, system, user
from sglang import image
from sglang import RuntimeEndpoint, set_default_backend
from sglang.srt.utils import load_image
from sglang.test.test_utils import is_in_ci
from sglang.utils import print_highlight, terminate_process, wait_for_server

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-05-01 02:12:21] server_args=ServerArgs(model_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-7B-Instruct', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=38377, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=832575969, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_l

[2025-05-01 02:12:30] Attention backend not set. Use fa3 backend by default.
[2025-05-01 02:12:30] Init torch distributed begin.


[2025-05-01 02:12:30] Init torch distributed ends. mem usage=0.00 GB
[2025-05-01 02:12:30] Load weight begin. avail mem=38.78 GB
[2025-05-01 02:12:31] Ignore import error when loading sglang.srt.models.llama4. 


[2025-05-01 02:12:31] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  1.58it/s]


...

[2025-05-01 02:12:35] INFO:     Started server process [933746]
[2025-05-01 02:12:35] INFO:     Waiting for application startup.
[2025-05-01 02:12:35] INFO:     Application startup complete.
[2025-05-01 02:12:35] INFO:     Uvicorn running on http://0.0.0.0:38377 (Press CTRL+C to quit)


[2025-05-01 02:12:35] INFO:     127.0.0.1:57768 - "GET /v1/models HTTP/1.1" 200 OK


[2025-05-01 02:12:36] INFO:     127.0.0.1:57782 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-01 02:12:36] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-01 02:12:38] INFO:     127.0.0.1:57786 - "POST /generate HTTP/1.1" 200 OK
[2025-05-01 02:12:38] The server is fired up and ready to roll!


Server started on http://localhost:38377


Set the default backend. Note: Besides the local server, you may use also `OpenAI` or other API endpoints.

In [2]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-05-01 02:12:40] INFO:     127.0.0.1:57792 - "GET /get_model_info HTTP/1.1" 200 OK


## Basic Usage

The most simple way of using SGLang frontend language is a simple question answer dialog between a user and an assistant.

In [3]:
@function
def basic_qa(s, question):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user(question)
    s += assistant(gen("answer", max_tokens=512))

In [4]:
state = basic_qa("List 3 countries and their capitals.")
print_highlight(state["answer"])

[2025-05-01 02:12:40] Prefill batch. #new-seq: 1, #new-token: 31, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-01 02:12:41] Decode batch. #running-req: 1, #token: 64, token usage: 0.00, gen throughput (token/s): 6.21, #queue-req: 0
[2025-05-01 02:12:41] INFO:     127.0.0.1:57796 - "POST /generate HTTP/1.1" 200 OK


## Multi-turn Dialog

SGLang frontend language can also be used to define multi-turn dialogs.

In [5]:
@function
def multi_turn_qa(s):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user("Please give me a list of 3 countries and their capitals.")
    s += assistant(gen("first_answer", max_tokens=512))
    s += user("Please give me another list of 3 countries and their capitals.")
    s += assistant(gen("second_answer", max_tokens=512))
    return s


state = multi_turn_qa()
print_highlight(state["first_answer"])
print_highlight(state["second_answer"])

[2025-05-01 02:12:41] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 18, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-01 02:12:41] INFO:     127.0.0.1:57800 - "POST /generate HTTP/1.1" 200 OK


[2025-05-01 02:12:41] Prefill batch. #new-seq: 1, #new-token: 23, #cached-token: 68, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-01 02:12:41] Decode batch. #running-req: 1, #token: 96, token usage: 0.00, gen throughput (token/s): 61.17, #queue-req: 0


[2025-05-01 02:12:42] INFO:     127.0.0.1:57804 - "POST /generate HTTP/1.1" 200 OK


## Control flow

You may use any Python code within the function to define more complex control flows.

In [6]:
@function
def tool_use(s, question):
    s += assistant(
        "To answer this question: "
        + question
        + ". I need to use a "
        + gen("tool", choices=["calculator", "search engine"])
        + ". "
    )

    if s["tool"] == "calculator":
        s += assistant("The math expression is: " + gen("expression"))
    elif s["tool"] == "search engine":
        s += assistant("The key word to search is: " + gen("word"))


state = tool_use("What is 2 * 2?")
print_highlight(state["tool"])
print_highlight(state["expression"])

[2025-05-01 02:12:42] Prefill batch. #new-seq: 1, #new-token: 25, #cached-token: 8, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-01 02:12:42] INFO:     127.0.0.1:38962 - "POST /generate HTTP/1.1" 200 OK
[2025-05-01 02:12:42] Prefill batch. #new-seq: 2, #new-token: 5, #cached-token: 62, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-01 02:12:42] INFO:     127.0.0.1:38978 - "POST /generate HTTP/1.1" 200 OK


[2025-05-01 02:12:42] Prefill batch. #new-seq: 1, #new-token: 13, #cached-token: 33, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-01 02:12:42] Decode batch. #running-req: 1, #token: 53, token usage: 0.00, gen throughput (token/s): 50.75, #queue-req: 0


[2025-05-01 02:12:43] INFO:     127.0.0.1:38984 - "POST /generate HTTP/1.1" 200 OK


## Parallelism

Use `fork` to launch parallel prompts. Because `sgl.gen` is non-blocking, the for loop below issues two generation calls in parallel.

In [7]:
@function
def tip_suggestion(s):
    s += assistant(
        "Here are two tips for staying healthy: "
        "1. Balanced Diet. 2. Regular Exercise.\n\n"
    )

    forks = s.fork(2)
    for i, f in enumerate(forks):
        f += assistant(
            f"Now, expand tip {i+1} into a paragraph:\n"
            + gen("detailed_tip", max_tokens=256, stop="\n\n")
        )

    s += assistant("Tip 1:" + forks[0]["detailed_tip"] + "\n")
    s += assistant("Tip 2:" + forks[1]["detailed_tip"] + "\n")
    s += assistant(
        "To summarize the above two tips, I can say:\n" + gen("summary", max_tokens=512)
    )


state = tip_suggestion()
print_highlight(state["summary"])

[2025-05-01 02:12:43] Prefill batch. #new-seq: 2, #new-token: 70, #cached-token: 28, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-01 02:12:43] Decode batch. #running-req: 2, #token: 64, token usage: 0.00, gen throughput (token/s): 69.06, #queue-req: 0


[2025-05-01 02:12:43] Decode batch. #running-req: 2, #token: 144, token usage: 0.01, gen throughput (token/s): 129.62, #queue-req: 0


[2025-05-01 02:12:44] Decode batch. #running-req: 2, #token: 224, token usage: 0.01, gen throughput (token/s): 128.57, #queue-req: 0


[2025-05-01 02:12:45] Decode batch. #running-req: 2, #token: 304, token usage: 0.01, gen throughput (token/s): 126.68, #queue-req: 0


[2025-05-01 02:12:45] INFO:     127.0.0.1:38994 - "POST /generate HTTP/1.1" 200 OK
[2025-05-01 02:12:45] Decode batch. #running-req: 1, #token: 214, token usage: 0.01, gen throughput (token/s): 115.31, #queue-req: 0


...

[2025-05-01 02:12:48] Decode batch. #running-req: 1, #token: 588, token usage: 0.03, gen throughput (token/s): 64.17, #queue-req: 0


[2025-05-01 02:12:49] Decode batch. #running-req: 1, #token: 628, token usage: 0.03, gen throughput (token/s): 63.94, #queue-req: 0


[2025-05-01 02:12:50] Decode batch. #running-req: 1, #token: 668, token usage: 0.03, gen throughput (token/s): 63.86, #queue-req: 0


[2025-05-01 02:12:50] Decode batch. #running-req: 1, #token: 708, token usage: 0.03, gen throughput (token/s): 63.85, #queue-req: 0


[2025-05-01 02:12:51] INFO:     127.0.0.1:39004 - "POST /generate HTTP/1.1" 200 OK


## Constrained Decoding

Use `regex` to specify a regular expression as a decoding constraint. This is only supported for local models.

In [8]:
@function
def regular_expression_gen(s):
    s += user("What is the IP address of the Google DNS servers?")
    s += assistant(
        gen(
            "answer",
            temperature=0,
            regex=r"((25[0-5]|2[0-4]\d|[01]?\d\d?).){3}(25[0-5]|2[0-4]\d|[01]?\d\d?)",
        )
    )


state = regular_expression_gen()
print_highlight(state["answer"])

[2025-05-01 02:12:51] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 12, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-01 02:12:52] Decode batch. #running-req: 1, #token: 37, token usage: 0.00, gen throughput (token/s): 21.76, #queue-req: 0
[2025-05-01 02:12:52] INFO:     127.0.0.1:39008 - "POST /generate HTTP/1.1" 200 OK


Use `regex` to define a `JSON` decoding schema.

In [9]:
character_regex = (
    r"""\{\n"""
    + r"""    "name": "[\w\d\s]{1,16}",\n"""
    + r"""    "house": "(Gryffindor|Slytherin|Ravenclaw|Hufflepuff)",\n"""
    + r"""    "blood status": "(Pure-blood|Half-blood|Muggle-born)",\n"""
    + r"""    "occupation": "(student|teacher|auror|ministry of magic|death eater|order of the phoenix)",\n"""
    + r"""    "wand": \{\n"""
    + r"""        "wood": "[\w\d\s]{1,16}",\n"""
    + r"""        "core": "[\w\d\s]{1,16}",\n"""
    + r"""        "length": [0-9]{1,2}\.[0-9]{0,2}\n"""
    + r"""    \},\n"""
    + r"""    "alive": "(Alive|Deceased)",\n"""
    + r"""    "patronus": "[\w\d\s]{1,16}",\n"""
    + r"""    "bogart": "[\w\d\s]{1,16}"\n"""
    + r"""\}"""
)


@function
def character_gen(s, name):
    s += user(
        f"{name} is a character in Harry Potter. Please fill in the following information about this character."
    )
    s += assistant(gen("json_output", max_tokens=256, regex=character_regex))


state = character_gen("Harry Potter")
print_highlight(state["json_output"])

[2025-05-01 02:12:53] Prefill batch. #new-seq: 1, #new-token: 24, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-01 02:12:53] Decode batch. #running-req: 1, #token: 70, token usage: 0.00, gen throughput (token/s): 35.89, #queue-req: 0


[2025-05-01 02:12:54] Decode batch. #running-req: 1, #token: 110, token usage: 0.01, gen throughput (token/s): 64.36, #queue-req: 0


[2025-05-01 02:12:54] Decode batch. #running-req: 1, #token: 1, token usage: 0.00, gen throughput (token/s): 64.18, #queue-req: 0
[2025-05-01 02:12:54] INFO:     127.0.0.1:59328 - "POST /generate HTTP/1.1" 200 OK


## Batching 

Use `run_batch` to run a batch of prompts.

In [10]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


states = text_qa.run_batch(
    [
        {"question": "What is the capital of the United Kingdom?"},
        {"question": "What is the capital of France?"},
        {"question": "What is the capital of Japan?"},
    ],
    progress_bar=True,
)

for i, state in enumerate(states):
    print_highlight(f"Answer {i+1}: {states[i]['answer']}")

[2025-05-01 02:12:54] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 13, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-01 02:12:54] INFO:     127.0.0.1:59342 - "POST /generate HTTP/1.1" 200 OK


  0%|          | 0/3 [00:00<?, ?it/s]

[2025-05-01 02:12:54] Prefill batch. #new-seq: 1, #new-token: 11, #cached-token: 17, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-01 02:12:54] Prefill batch. #new-seq: 2, #new-token: 18, #cached-token: 34, token usage: 0.00, #running-req: 1, #queue-req: 0


 33%|███▎      | 1/3 [00:00<00:00,  6.00it/s]

 67%|██████▋   | 2/3 [00:00<00:00,  9.52it/s]

100%|██████████| 3/3 [00:00<00:00, 15.19it/s]

[2025-05-01 02:12:55] INFO:     127.0.0.1:59354 - "POST /generate HTTP/1.1" 200 OK
[2025-05-01 02:12:55] INFO:     127.0.0.1:59356 - "POST /generate HTTP/1.1" 200 OK
[2025-05-01 02:12:55] INFO:     127.0.0.1:59352 - "POST /generate HTTP/1.1" 200 OK


## Streaming 

Use `stream` to stream the output to the user.

In [11]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


state = text_qa.run(
    question="What is the capital of France?", temperature=0.1, stream=True
)

for out in state.text_iter():
    print(out, end="", flush=True)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What is the capital of France?<|im_end|>
<|im_start|>assistant


[2025-05-01 02:12:55] INFO:     127.0.0.1:59358 - "POST /generate HTTP/1.1" 200 OK
[2025-05-01 02:12:55] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 25, token usage: 0.00, #running-req: 0, #queue-req: 0
The

 capital

 of

 France

 is

 Paris

.

<|im_end|>


## Complex Prompts

You may use `{system|user|assistant}_{begin|end}` to define complex prompts.

In [12]:
@function
def chat_example(s):
    s += system("You are a helpful assistant.")
    # Same as: s += s.system("You are a helpful assistant.")

    with s.user():
        s += "Question: What is the capital of France?"

    s += assistant_begin()
    s += "Answer: " + gen("answer", max_tokens=100, stop="\n")
    s += assistant_end()


state = chat_example()
print_highlight(state["answer"])

[2025-05-01 02:12:55] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-01 02:12:55] INFO:     127.0.0.1:59362 - "POST /generate HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

[2025-05-01 02:12:55] Child process unexpectedly failed with an exit code 9. pid=934472
[2025-05-01 02:12:55] Child process unexpectedly failed with an exit code 9. pid=934329


## Multi-modal Generation

You may use SGLang frontend language to define multi-modal prompts.
See [here](https://docs.sglang.ai/supported_models/generative_models.html) for supported models.

In [14]:
server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-VL-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-05-01 02:13:04] server_args=ServerArgs(model_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-VL-7B-Instruct', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=37391, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=956388408, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step

[2025-05-01 02:13:06] Infer the chat template name from the model path and obtain the result: qwen2-vl.


[2025-05-01 02:13:15] Attention backend not set. Use flashinfer backend by default.
[2025-05-01 02:13:15] Automatically reduce --mem-fraction-static to 0.792 because this is a multimodal model.
[2025-05-01 02:13:15] Automatically turn off --chunked-prefill-size for multimodal model.
[2025-05-01 02:13:15] Init torch distributed begin.


[2025-05-01 02:13:15] Init torch distributed ends. mem usage=0.00 GB
[2025-05-01 02:13:15] Load weight begin. avail mem=61.90 GB
[2025-05-01 02:13:16] Ignore import error when loading sglang.srt.models.llama4. 


[2025-05-01 02:13:17] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]


...

[2025-05-01 02:13:23] INFO:     127.0.0.1:44154 - "GET /v1/models HTTP/1.1" 200 OK


[2025-05-01 02:13:23] INFO:     127.0.0.1:44166 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-01 02:13:24] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


2025-05-01 02:13:25,869 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False
2025-05-01 02:13:25,987 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


[2025-05-01 02:13:26] INFO:     127.0.0.1:44180 - "POST /generate HTTP/1.1" 200 OK
[2025-05-01 02:13:26] The server is fired up and ready to roll!


Server started on http://localhost:37391


In [15]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-05-01 02:13:28] INFO:     127.0.0.1:44192 - "GET /get_model_info HTTP/1.1" 200 OK


Ask a question about an image.

In [16]:
@function
def image_qa(s, image_file, question):
    s += user(image(image_file) + question)
    s += assistant(gen("answer", max_tokens=256))


image_url = "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
image_bytes, _ = load_image(image_url)
state = image_qa(image_bytes, "What is in the image?")
print_highlight(state["answer"])

[2025-05-01 02:13:29] Prefill batch. #new-seq: 1, #new-token: 307, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-01 02:13:30] Decode batch. #running-req: 1, #token: 340, token usage: 0.02, gen throughput (token/s): 5.01, #queue-req: 0


[2025-05-01 02:13:31] Decode batch. #running-req: 1, #token: 380, token usage: 0.02, gen throughput (token/s): 38.25, #queue-req: 0


[2025-05-01 02:13:32] Decode batch. #running-req: 1, #token: 420, token usage: 0.02, gen throughput (token/s): 38.69, #queue-req: 0
[2025-05-01 02:13:32] INFO:     127.0.0.1:44194 - "POST /generate HTTP/1.1" 200 OK


In [17]:
terminate_process(server_process)

[2025-05-01 02:13:32] Child process unexpectedly failed with an exit code 9. pid=936323
